In [1]:
# _*_ EG1. exec(str, globals, locals) _*_ #
#可以把exec命令的执行当成是一个函数的执行，会将执行期间产生的名字存放于局部名称空间中
g={
    'x':1,
    'y':2
}
l={}
 
exec('''
global x,z
x=100
z=200
 
m=300
''',g,l)
 
print(g) #{'x': 100, 'y': 2,'z':200,......}
print(l) #{'m': 300}

{'x': 100, 'y': 2, '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod': <built-in function divmod>, 'eval': <built-in function eval>, 'exec': <built-in function exec>, 'format': <built-in fu

In [2]:
# _*_ EG2. 类也是对象 _*_ #
# 1.type 函数可以查看类型，也可以用来查看对象所属的类
print(type(list))

# 2.type 函数还可以动态创建类
'''
type(name of the class,
     tuple of the parent class (for inheritance, can be empty),
     dictionary containing attributes names and values)
'''
type('FooChild', (Foo,), {'echo_bar': echo_bar}) 
<==>
class FooChild(Foo):
    def echo_bar:  pass
    
# 3.元类是类的模板，元类 metaclass 的实例化 -> 类class，类class的实例化 -> 对象object
# ype是python的一个内建元类，用来直接控制生成类，python中任何class定义的类其实都是type类实例化的对象

<class 'type'>


In [10]:
# _*_ EG3. 类的元类探索 _*_ #

class Mymeta(type):
 
    def __init__(self,class_name,class_bases,class_dic):
        if not class_name.istitle():
            raise TypeError('类名的首字母必须大写')
        if '__doc__' not in class_dic or not class_dic['__doc__'].strip():
            raise TypeError('必须有注释，且注释不能为空')
        super(Mymeta,self).__init__(class_name,class_bases,class_dic)

class Chinese(object, metaclass=Mymeta):
    '''
    有注释
    '''
    country = 'china'
 
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def talk(self):
        print('%s is talking'%self.name)
        
print(Chinese.__doc__)
print(Chinese.__class__)

age,name = 35, 'bob'
print(age.__class__) # == type(age)
print(name.__class__)# == type(name)
print(age.__class__.__class__)
print(name.__class__.__class__)

print(type(name.__class__))


    有注释
    
<class '__main__.Mymeta'>
<class 'int'>
<class 'str'>
<class 'type'>
<class 'type'>
<class 'type'>


In [9]:
# _*_ EG4. 手动模拟类的创建过程 _*_ #

class_name = 'Chinese'
class_bases = (object, )
class_body = """
country = 'China'
def __init__(self, name, age):
    self.name, self.age = name, age
def talk(self):
    print('%s is talking' % self.name)
"""

# step 1:（先处理类体->名称空间）：
# 类体定义的名字都会存放于类的名称空间中（一个局部的名称空间），
# 可以事先定义一个空字典，然后用exec去执行类体的代码（exec产生名称空间的过程与真正的class过程类似，只是后者会将__开头的属性变形），生成类的局部名称空间，即填充字典
class_dic = {}
exec(class_body, globals(), class_dic)
print(class_dic)

# step 2: 调用元类 type
Foo = type(class_name, class_bases, class_dic)
print(Foo)
print(type(Foo))
print(isinstance(Foo, type))

{'country': 'China', '__init__': <function __init__ at 0x000001ACCA3269D0>, 'talk': <function talk at 0x000001ACCA326820>}
<class '__main__.Chinese'>
<class 'type'>
True


In [ ]:
# _*_ EG5. 自定义元类，控制类的行为 _*_ #
# 元类的目的：创建类时，做一些自动化的处理、初始化

# eg1. 使得类的所有属性初始字符大写
class UpperAttrMetaclass(type):
    # __new__ 方法在 __init__ 前执行，这个方法创建对象并返回。
    # __init__方法仅仅初始化作为参数传入的对象。
    # 你很少使用__new__方法，除非你想控制类是如何创建的。
    # 这里创建的对象是一个类，我们想自定义它，因此需要覆盖 __new__。
    # 你也能在__init__中做一些事情，如果愿意的话。
    # 一些高级用法还包括覆盖__call__方法，这里不使用。
    def __new__(upperattr_metaclass, future_class_name,
                future_class_parents, future_class_attr):
 
        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val
 
        return type(future_class_name, future_class_parents, uppercase_attr)


# eg2. 上例中并未覆盖或者调用type.__new__，显得不够OOP
# 参数upperattr_metaclass，事实上：__new__方法总是接受定义的类作为第一个参数，就像初始化实例时第一个参数传入self
class UpperAttrMetaclass(type):
 
    def __new__(upperattr_metaclass, future_class_name,
                future_class_parents, future_class_attr):
 
        uppercase_attr = {}
        for name, val in future_class_attr.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val
 
        # 重用了type.__new__方法，这种写法是OOP的
        return type.__new__(upperattr_metaclass, future_class_name,
                            future_class_parents, uppercase_attr)
    

# eg3. 更标准一些的写法
class UpperAttrMetaclass(type):
 
    def __new__(cls, clsname, bases, dct):
 
        uppercase_attr = {}
        for name, val in dct.items():
            if not name.startswith('__'):
                uppercase_attr[name.upper()] = val
            else:
                uppercase_attr[name] = val
 
        return super(UpperAttrMetaclass, cls).__new__(cls, clsname, bases, uppercase_attr)

In [13]:
# 在元类中控制把自定义类的数据属性都变成大写
class Mymetaclass(type):
    def __new__(cls,name,bases,attrs):
        update_attrs={}
        for k,v in attrs.items():
            if not callable(v) and not k.startswith('__'):
                update_attrs[k.upper()]=v
            else:
                update_attrs[k]=v
        return type.__new__(cls,name,bases,update_attrs)
 
class Chinese(metaclass=Mymetaclass):
    country='China'
    tag='Legend of the Dragon' #龙的传人
    def walk(self):
        print('%s is walking' %self.name)
 
 
print(Chinese.__dict__)

# 在元类中控制自定义的类无需__init__方法
'''
1.元类帮其完成创建对象，以及初始化操作；
2.要求实例化时传参必须为关键字形式，否则抛出异常TypeError: must use keyword argument
3.key作为用户自定义类产生对象的属性，且所有属性变成大写
'''
class Mymetaclass(type):
    # def __new__(cls,name,bases,attrs):
    #     update_attrs={}
    #     for k,v in attrs.items():
    #         if not callable(v) and not k.startswith('__'):
    #             update_attrs[k.upper()]=v
    #         else:
    #             update_attrs[k]=v
    #     return type.__new__(cls,name,bases,update_attrs)
 
    def __call__(self, *args, **kwargs):
        if args:
            raise TypeError('must use keyword argument for key function')
        obj = object.__new__(self) #创建对象，self为类Foo
 
        for k,v in kwargs.items():
            obj.__dict__[k.upper()]=v
        return obj
 
class Chinese(metaclass=Mymetaclass):
    country='China'
    tag='Legend of the Dragon' #龙的传人
    def walk(self):
        print('%s is walking' %self.name)
 
 
p=Chinese(name='egon',age=18,sex='male')
print(p.__dict__)

{'__module__': '__main__', 'COUNTRY': 'China', 'TAG': 'Legend of the Dragon', 'walk': <function Chinese.walk at 0x000001ACCA326700>, '__dict__': <attribute '__dict__' of 'Chinese' objects>, '__weakref__': <attribute '__weakref__' of 'Chinese' objects>, '__doc__': None}
{'NAME': 'egon', 'AGE': 18, 'SEX': 'male'}
